# Properly Ordered Card Hands

The 538 Riddler [presented](https://fivethirtyeight.com/features/who-will-capture-the-most-james-bonds/) this problem by Matt Ginsberg:
    
> *You play so many card games that you’ve developed a very specific organizational obsession. When you’re dealt your hand, you want to organize it such that the cards of a given suit are grouped together and, if possible, such that no suited groups of the same color are adjacent. (Numbers don’t matter to you.) Moreover, when you receive your randomly ordered hand, you want to achieve this organization with a single motion, moving only one adjacent block of cards to some other position in your hand, maintaining the original order of that block and other cards, except for that one move.*

> *Suppose you’re playing pitch, in which a hand has six cards. What are the odds that you can accomplish your obsessive goal? What about for another game, where a hand has N cards, somewhere between 1 and 13?*

# Complexity

The first thing to decide is how many `N`-card hands are there? That will tell if I can just use brute force.

The answer is (52 choose `N`), and (52 choose 6) is 20,358,520. So it is barely feasible to use brute force there. But I notice that the problem states *"Numbers don’t matter,"* so I can just consider the *suits* of the cards. Then there are only 4<sup>`N`</sup> hands, which is a mere 4,096 for `N` = 6, and a barely feasible 67,108,864 for `N` = 13.

# Deals: Hands and their Probabilities

There are two red suits and two black suits, so I'll represent the four suits with the characters `'rbRB'`. (I also considered using `'♠️♥️♦️♣️'`.) I'll represent a hand as a string of suits: `'rrBrbr'` is a 6-card hand. I'll define `deals(N)` to return a dict of all possible hands of length `N`, each mapped to the probability of the hand. I'll use exact `Fraction` arithmetic. I'll use `lru_cache` when there are expensive computations that I don't want to repeat.

In [1]:
import re
from collections import defaultdict, Counter
from fractions import Fraction
from functools import lru_cache

one   = Fraction(1)
suits = 'rbRB'

@lru_cache()
def deals(N):
    "A dict of {'chars': probability} for all hands of length N."
    if N == 0:
        return {'': one}
    else:
        return {hand + suit: p * (13 - hand.count(suit)) / (52 - len(hand))
                for (hand, p) in deals(N - 1).items()
                for suit in suits}

In [2]:
deals(2)

{'BB': Fraction(1, 17),
 'BR': Fraction(13, 204),
 'Bb': Fraction(13, 204),
 'Br': Fraction(13, 204),
 'RB': Fraction(13, 204),
 'RR': Fraction(1, 17),
 'Rb': Fraction(13, 204),
 'Rr': Fraction(13, 204),
 'bB': Fraction(13, 204),
 'bR': Fraction(13, 204),
 'bb': Fraction(1, 17),
 'br': Fraction(13, 204),
 'rB': Fraction(13, 204),
 'rR': Fraction(13, 204),
 'rb': Fraction(13, 204),
 'rr': Fraction(1, 17)}

Is that right? Yes it is. The probability of `'BB'` is 1/17, beause the probability of the first `'B'` is 13/52 or 1/4, and when we deal the second card, one `'B'` is gone, so the probability is 12/51, so that simplifies to 1/4 &times; 12/51 = 3/51 = 1/17. The probability of `'BR'` is 13/204, because the probability of the `'R'` is 13/51, and 1/4 &times; 13/51 = 13/204.

# Collapsing Hands

I'll introduce the idea of *collapsing* a hand by replacing a run of cards of the same suit with a single card, so that: 

     collapse('BBBBBrrrrBBBB') == 'BrB'
     
I'll use the term *hand* for `'BBBBBrrrrBBBB'`, and *sequence* or *seq* for the collapsed version, `'BrB'`.

# Properly Ordered Hands

A hand is considered properly `ordered` if *"the cards of a given suit are grouped together and, if possible, such that no suited groups of the same color are adjacent."* I was initially confused about the meaning of *"if possible";* Matt Ginsberg confirmed it means that the hand `'BBBbbb'` is properly ordered, because it is not possible to separate the two black suits, while `'BBBbbR'` is not properly ordered, because the red card could have been inserted between the two black runs.

So a hand is properly ordered if, considering its collapsed sequence, each suit appears only once, and either all the colors are the same, or suits of the same color don't appear adjacent to each other.

In [3]:
def ordered(hand):
    "Properly ordered if each suit run appears only once, and same color suits not adjacent."
    seq = collapse(hand)
    return once_each(seq) and (len(colors(seq)) == 1 or not adjacent_colors(seq))
                                 
def collapse(hand): return re.sub(r'(.)\1+', r'\1', hand)
def once_each(seq): return max(Counter(seq).values()) == 1
def colors(seq):    return set(seq.casefold())
adjacent_colors =   re.compile('rR|Rr|Bb|bB').search

In [4]:
collapse('BBBBBrrrrBBBB')

'BrB'

In [5]:
ordered('BBBbbR') 

False

# Moving Cards to Make a Hand Ordered

I won't try to be clever; I'm content to use brute force. I'll say that a collapsed sequence is `orderable` if any of the possible `moves` of a block of cards makes the hand `ordered`. I'll find all possible `moves`, by finding all possible  `splits` of the cards into a middle block of cards flanked by (possibly empty) left and right sequences; then all possible `inserts` of the block back into the rest of the cards.  I'll define `orderable_probability(N)` to give the probability that a random `N`-card hand is orderable.
Since many hands will collapse to the same sequence, I'll throw a `lru_cache` onto `orderable` so that it won't have to repeat computations.

In [6]:
@lru_cache(None)
def orderable(seq): return any(ordered(m) for m in moves(seq))

def orderable_probability(N):
    "What's the probability that an N-card hand is orderable?"
    return sum(p for (hand, p) in deals(N).items() if orderable(collapse(hand)))

def moves(seq):
    "All possible ways of moving a single block of cards."
    return {collapse(s) for (L, M, R) in splits(seq)
            for s in inserts(M, L + R)}

def inserts(block, others):
    "All ways of inserting a block into the other cards."
    return [others[:i] + block + others[i:]
            for i in range(len(others) + 1)]

def splits(seq):
    "All ways of splitting a hand into a non-empty middle flanked by left and right parts."
    return [(seq[:i], seq[i:j], seq[j:])
            for i in range(len(seq))
            for j in range(i + 1, len(seq) + 1)]

# First Answer

Here's our answer for 6 cards:

In [7]:
orderable_probability(6)

Fraction(51083, 83895)

And an easier-to-read answer for everthing up to `N` = 7 cards:

In [8]:
def report(Ns):
    "Show the probability of orderability, for each N in Ns."
    for N in Ns:
        P = orderable_probability(N)
        print('{:2}: {:6.1%} = {}'.format(N, float(P), P))
        
report(range(1, 8))

 1: 100.0% = 1
 2: 100.0% = 1
 3: 100.0% = 1
 4: 100.0% = 1
 5:  85.2% = 213019/249900
 6:  60.9% = 51083/83895
 7:  37.3% = 33606799/90047300


# Getting to `N` = 13

That looks good, but if we want to get to 13-card hands, we'll have to handle 4<sup>13</sup> = 67,108,864 `deals`, which will take a long time. But I have an idea to speed things up: Consider the sequence `'rbrRrBbRB'`. It has 9 runs, and the most a properly ordered hand can have is 4 runs. What's the most number of runs that can be reduced by a singe move? One run could be reduced when we remove  block, if the cards on either side of the block are the same. When we replace the block, we can reduce 2 more, if the left and right ends of the block match the cards to the left and right of the new position. So that makes 3. Therefore, we can skip creating any hand with more than 7 runs. I will modify `deals(N)` to drop any such hands.

Here's an example of a moving a block [bracketed] to reduce the number of runs from 7 to 4:

       bRB[bR]Br   =>   b[bR]RBBr  =   bRBr
    

In [9]:
@lru_cache()
def deals(N):
    "A dict of {'chars': probability} for all hands of length N with under 8 runs."
    if N == 0:
        return {'': one}
    else:
        return {hand + suit: p * (13 - hand.count(suit)) / (52 - len(hand))
                for (hand, p) in deals(N - 1).items()
                for suit in suits
                if len(collapse(hand + suit)) <= 7} # <<<< CHANGE HERE

# Final Answer

We're finaly ready to go up to `N` = 13. This will take several minutes:

In [10]:
%time report(range(1, 14))

 1: 100.0% = 1
 2: 100.0% = 1
 3: 100.0% = 1
 4: 100.0% = 1
 5:  85.2% = 213019/249900
 6:  60.9% = 51083/83895
 7:  37.3% = 33606799/90047300
 8:  20.2% = 29210911/144718875
 9:   9.9% = 133194539/1350709500
10:   4.4% = 367755247/8297215500
11:   1.9% = 22673450197/1219690678500
12:   0.7% = 1751664923/238130084850
13:   0.3% = 30785713171/11112737293000
CPU times: user 3min 52s, sys: 3.48 s, total: 3min 55s
Wall time: 4min 8s


It certainly is encouraging that, for everything up to `N` = 7,  we get the same answers as the previous `report`.

# Unit Tests

To gain confidence in these answers, here are some unit tests. Before declaring my answers definitively correct, I would want a lot more tests, and some independent code reviews.

In [11]:
def test():
    assert deals(1) == {'B': 1/4, 'R': 1/4, 'b': 1/4, 'r': 1/4}
    assert len(deals(6)) == 4 ** 6
    assert ordered('BBBBBrrrrBBBB') is False
    assert ordered('BBBBBrrrrRRRR') is False
    assert ordered('BBBbbr') is False # Bb
    assert ordered('BBBbbrB') is False # two B's
    assert ordered('BBBbbb') 
    assert ordered('BBBbbbB') is False # two B's
    assert ordered('BBBBBrrrrbbbb')
    assert colors('BBBBBrrrrbbbb') == {'r', 'b'}
    assert once_each('Bb')
    assert once_each('BbR')
    assert adjacent_colors('BBBbbR')
    assert not adjacent_colors('BBBBBrrrrBBBB')
    assert collapse('BBBBBrrrrBBBB') == 'BrB'
    assert collapse('brBBrrRR') == 'brBrR'
    assert collapse('bbbbBBBrrr') == 'bBr'
    assert moves('bRb') == {'Rb', 'bR', 'bRb'}
    assert moves('bRBb') == {
        'BbR', 'BbRb', 'RBb', 'RbBb', 'bBRb', 'bBbR', 'bRB', 'bRBb', 'bRbB'}
    assert inserts('BB', '....') == [
        'BB....', '.BB...', '..BB..', '...BB.', '....BB']
    assert splits('123') == [('', '1', '23'), ('', '12', '3'), ('', '123', ''),
                             ('1', '2', '3'), ('1', '23', ''), ('12', '3', '')]
    assert orderable('bBr') # move 'r' after 'b'
    assert orderable('bBrbRBr') # move 'bRB' after first 'b' to get 'bbRBBrr'
    assert orderable('bBrbRBrb') is False
    return True

test()

True

# Table Size

A key function in this program is `orderable(seq)`. Let's look at its cache:

In [12]:
orderable.cache_info()

CacheInfo(hits=7198870, misses=4373, maxsize=None, currsize=4373)

So we looked at over 7 million hands, but only 4373 different collapsed sequences. And once we hit `N` = 7, we've seen all the sequences we're ever going to see. From `N` = 8 and up, almost all the computation goes into computing the probability of each hand, not into deciding the orderability of each sequence.